### Checkpoint 6

In [4]:
# all required imports
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit as im
from scipy.special import erf


#constants
F = 0.98
A = 5
M = 2.5
W = 0.2
XMIN = 0
XMAX = 10
NEVENTS = 10000

In [5]:
#importing required files
mass_dis = np.loadtxt('datafile-higgs.txt')

## Part 1

Defining the NLL function

In [6]:
def nll_gaussian(M,W):
    """NLL function for Gaussian distribution

    Args:
        M (float): Mean of the Gaussian Signal 
        W (float): Width of the Gaussian PDF
    Returns: 
        float: NLL function 
    """
    s = W/2 # Gaussian half-width. To reuse older equations
    n_g = 1.25331*s*(erf(0.707107*M/s)-erf(0.707107*(M-10)/s)) # normalization for the gaussian pdf
    return  np.exp(-0.5*((mass_dis-M)/s)**2)*(1/n_g) # gaussian pdf


def nll_exp(A):
    """NLL function for Exponential distribution

    Args:
        A (float): exponential parameter A
    Returns: 
        float: NLL function
    """
    
    n_exp   = (1-np.exp(-10/A))*A  # normalization for the exponential pdf
    return (1/n_exp)*np.exp(-mass_dis/A) #exponential pdf


def nll_g_e(M,W,F,A):
    """Generates combined NLL function from the sum of Exponential and Gaussian PDFs normalised in the range [0,10]

    Args:
        M (float): Mean of the Gaussian Signal 
        A (float): exponential parameter A
        W (float): Width of the Gaussian PDF
        F (float): fraction of counts in the Exponential PDF

    Returns:
        float : NLL function
    """
    return -np.sum(np.log(F*nll_exp(A) + (1-F)*nll_gaussian(M,W))) #combined nll

Setting up and finding the best parameters using iMinuit

In [7]:
m = im(nll_g_e,M=M,W=W,F=F,A=A)
m.errordef = 0.5 #im.LIKELIHOOD
#m.fixed['W'] = True 
m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.146e+05                  │              Nfcn = 110              │
│ EDM = 9.22e-06 (Goal: 0.0001)    │            time = 0.4 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ M    │   2.499   │   0.014   │            │            │         │         │       │
│ 1 │ W    │   0.391   │   0.028   │            │            │         │         │       │
│ 2 │ F    │  0.9799   │  0.0014   │            │            │         │         │       │
│ 3 │ A    │   5.007   │   0.031   │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬─────────────────────────────────────────┐
│   │         M         W         F         A │
├───┼─────────────────────────────────────────┤
│ M │   0.00021 -4.95e-05  1.96e-06 -2.11e-05 │
│ W │ -4.95e-05  0.000804 -2.09e-05  7.65e-05 │
│ F │  1.96e-06 -2.09e-05  1.84e-06 -5.89e-06 │
│ A │ -2.11e-05  7.65e-05 -5.89e-06  0.000951 │
└───┴─────────────────────────────────────────┘

In [8]:
M_est,W_est,F_est,A_est, = m.values[0:]
M_err,W_err,F_err,A_err = m.errors[0:]
print("The Gaussian Mean(M): {:.3f} +/- {:.3f}".format(M_est,M_err))
print("Exponential Parameter(A): {:.3f} +/- {:.3f}".format(A_est,A_err))
print("Fraction F: {:.4f} +/- {:.4f}".format(F_est,F_err)) #required

The Gaussian Mean(M): 2.499 +/- 0.015
Exponential Parameter(A): 5.007 +/- 0.031
Fraction F: 0.9799 +/- 0.0014


Required obtained best parameters.

    The Gaussian Mean(M): 2.499 +/- 0.015
    Exponential Parameter(A): 5.007 +/- 0.031
    Fraction F: 0.9799 +/- 0.0014

## Part 2

In [9]:
def nll_poly(a,b,c):
    """NLL function for Polynomial Distribution
        b/a and c/a from the question are redefined as b and c.

    Args:
       a,b,c (float): polynomial constants
    Returns:
        float: NLL function
    """
    
    n_poly = (10/3)*a*(15*b+100*c+3)
    return a*(1+b*mass_dis+c*mass_dis**2)/n_poly

def nll_g_poly(M,W,F,a,b,c):
    """Generates combined NLL function from the sum of Exponential and Gaussian PDFs normalised in the range [0,10]

    Args:
        M (float): Mean of the Gaussian Signal 
        W (float): Width of the Gaussian PDF
        F (float): fraction of counts of the background 
        a,b,c (float): polynomial constants

    Returns:
        float : NLL function
    """
    return -np.sum(np.log(F*nll_poly(a,b,c) + (1-F)*nll_gaussian(M,W))) #combined nll

In [10]:
m_poly_backgorund = im(nll_g_poly,M=M,W=W,F=F,a=1,b=-0.16,c=0.007)
m_poly_backgorund.fixed['a'] = True
m_poly_backgorund.errordef = 0.5 #im.LIKELIHOOD
m_poly_backgorund.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.146e+05                  │              Nfcn = 179              │
│ EDM = 2.4e-05 (Goal: 0.0001)     │            time = 0.7 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ M    │   2.498   │   0.015   │            │            │         │         │       │
│ 1 │ W    │   0.350   │   0.027   │            │            │         │         │       │
│ 2 │ F    │  0.9832   │  0.0013   │            │            │         │         │       │
│ 3 │ a    │   1.00    │   0.01    │            │            │         │         │  yes  │
│ 4 │ b    │  -0.1579  │  0.0011   │            │            │         │         │       │
│ 5 │ c    │  7.30e-3  │  0.12e-3  │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬─────────────────────────────────────────────────────────────┐
│   │         M         W         F         a         b         c │
├───┼─────────────────────────────────────────────────────────────┤
│ M │   0.00022 -5.32e-05  1.87e-06         0  -5.2e-07  4.76e-08 │
│ W │ -5.32e-05    0.0007 -1.69e-05         0  5.81e-07  1.53e-10 │
│ F │  1.87e-06 -1.69e-05  1.59e-06         0 -4.31e-08 -6.37e-10 │
│ a │         0         0         0         0         0         0 │
│ b │  -5.2e-07  5.81e-07 -4.31e-08         0  1.13e-06 -1.23e-07 │
│ c │  4.76e-08  1.53e-10 -6.37e-10         0 -1.23e-07  1.39e-08 │
└───┴─────────────────────────────────────────────────────────────┘

In [17]:
M_est_poly = m_poly_backgorund.values[0:][0]
M_err_poly = m.errors[0:][0]
Esys1 = abs(M_est_poly-M_est)
Etot1 = np.linalg.norm([Esys1,M_err])

print("The Gaussian Mean(M): {:.3f} +/- {:.3f} +/- {:.5f}".format(M_est, M_err, Esys1))
print("The Gaussian Mean(M): {:.3f} +/- {:.3f}".format(M_est,Etot1))


The Gaussian Mean(M): 2.499 +/- 0.015 +/- 0.00075
The Gaussian Mean(M): 2.499 +/- 0.015


    The Gaussian Mean(M): 2.499 +/- 0.015 +/- 0.00075
    The Gaussian Mean(M): 2.499 +/- 0.015
The Systematic error estimated from the shift between Polynomial background and Exponential background is insignificant(to the 2 significant figures) compared to the systematic error. In this case, we can easily neglect it. However, this is only done on one single dataset and with only one alternative model. The actual systematic error is still inconclusive from this.

## Part 3

In [12]:
def nll_linear(a,b):
    """NLL function for the Linear distribution

    Args:
        a (float): intercept
        b (float): slope/intercept

    Returns:
        float : NLL function
    """
    n_linear = 10*(a*b*5+a)
    return a*(1+b*mass_dis)/n_linear

def nll_g_linear(M,W,F,a,b):
    """Generates combined NLL function from the sum of Exponential and Gaussian PDFs normalised in the range [0,10]

    Args:
        M (float): Mean of the Gaussian Signal 
        W (float): Width of the Gaussian PDF
        F (float): fraction of counts of the background 
        a,b (float): linear constants

    Returns:
        float : NLL function
    """
    return -np.sum(np.log(F*nll_linear(a,b) + (1-F)*nll_gaussian(M,W))) #combined nll

In [13]:
m_linear_backgorund = im(nll_g_linear,M=M,W=W,F=F,a=1,b=-0.09)
m_linear_backgorund.fixed['a'] = True
m_linear_backgorund.errordef = 0.5 #im.LIKELIHOOD
m_linear_backgorund.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.155e+05                  │              Nfcn = 123              │
│ EDM = 3.62e-05 (Goal: 0.0001)    │            time = 0.4 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ M    │   2.465   │   0.016   │            │            │         │         │       │
│ 1 │ W    │   0.400   │   0.032   │            │            │         │         │       │
│ 2 │ F    │  0.9801   │  0.0014   │            │            │         │         │       │
│ 3 │ a    │   1.00    │   0.01    │            │            │         │         │  yes  │
│ 4 │ b    │ -90.40e-3 │  0.22e-3  │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬───────────────────────────────────────────────────┐
│   │         M         W         F         a         b │
├───┼───────────────────────────────────────────────────┤
│ M │  0.000265 -0.000191   5.5e-06         0 -1.98e-07 │
│ W │ -0.000191   0.00102 -2.61e-05         0  7.86e-07 │
│ F │   5.5e-06 -2.61e-05  2.02e-06         0 -5.37e-08 │
│ a │         0         0         0         0         0 │
│ b │ -1.98e-07  7.86e-07 -5.37e-08         0  5.03e-08 │
└───┴───────────────────────────────────────────────────┘

In [14]:
M_est_linear = m_linear_backgorund.values[0:][0]
M_err_linear = m.errors[0:][0]
Esys2 = abs(M_est_linear-M_est)
Etot2 = np.linalg.norm([Esys2,M_err])

print("The Gaussian Mean(M): {:.3f} +/- {:.3f} +/- {:.5f}".format(M_est, M_err, Esys2))
print("The Gaussian Mean(M): {:.3f} +/- {:.3f}".format(M_est, Etot2))


The Gaussian Mean(M): 2.499 +/- 0.015 +/- 0.03369
The Gaussian Mean(M): 2.499 +/- 0.037


The systematic error induced from the Linear background estimate is higher than that from the statistical error and contributes significantly to this total error.

The combined error using shift from both linear and polynomial background models is given below. Our estimation of systematic error is still incomplete because we did not do any estimation for the Peak in the dataset.

In [15]:
Etot = np.linalg.norm([Esys1,Esys2,M_err])
print("The Gaussian Mean(M): {:.3f} +/- {:.3f}".format(M_est,Etot))

The Gaussian Mean(M): 2.499 +/- 0.037
